In [3]:
from vpolo.alevin import parser
import scanpy as sc
import pandas as pd
import os 
from scipy.io import mmwrite
from scipy.sparse import csr_matrix
import tempfile

/opt/conda/envs/spatialdata/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/opt/conda/envs/spatialdata/lib/python3.10/site-packages/anndata/utils.py:429: Fut

In [8]:
def alevin_to_scanpy(Dataset_path):

    if os.path.isdir(Dataset_path):
        alevin_df = parser.read_quants_bin(Dataset_path).T
        #alevin_df = alevin_df.T
        print(alevin_df.shape)
        adata = sc.AnnData(alevin_df)
        print(adata)
        with tempfile.TemporaryDirectory() as tempdir:
            # matrix.mtx
            mmwrite(os.path.join(tempdir, "matrix.mtx"), csr_matrix(adata.X))
            print(adata.X)
            
            # barcode file
            barcodes_df = pd.DataFrame(adata.var_names) #not adata.obs_names as we are using transpose matrix
            barcodes_df.to_csv(os.path.join(tempdir, "barcodes.tsv"), sep="\t", index=False, header=False)
            print(adata.var_names)
            
            #adata.obs_names = adata.obs_names.str.split('.').str[0]
            # genes file
            genes_df = pd.DataFrame(adata.obs_names)
            genes_df.to_csv(os.path.join(tempdir, "genes.tsv"), sep="\t", index=False, header=False)
            print(genes_df)
            
            # scanpy input
            adata = sc.read_10x_mtx(
                tempdir,  # the directory with all three files
                var_names="gene_ids",  # use gene symbols for the variable names (variables-axis index)
                cache=False,  # disabling the automatic cache file so we can create one mannually in the training folder.
            )
            adata.var_names_make_unique()
        return adata

In [9]:
Dataset_path = "/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/align/SRR9036396"

output = alevin_to_scanpy(Dataset_path)

/home/jovyan/ifbdata/spatial_cell_id/Kush/alignment/demo_matrix/align/SRR9036396/alevin
Using rust mode with 3281 rows and 78277 columns
(78277, 3281)
AnnData object with n_obs × n_vars = 78277 × 3281
[[  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           1.5          0.         ...   0.33333334   0.
    0.        ]
 ...
 [162.         189.          55.         ...   8.           4.
    2.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]
 [  0.           0.           0.         ...   0.           0.
    0.        ]]
Index(['CTTAACTTCTGAGGGA', 'AACACGTGTACAAGTA', 'AGAGCTTTCTTAACCT',
       'TGGTTAGAGCAGACTG', 'CGGACGTCAACTGGCC', 'CGTTCTGGTTCGTTGA',
       'ATTGGACAGCGATTCT', 'GCGCAGTGTATATCCG', 'TGGACGCGTCCTGCTT',
       'CAGCTGGGTCCGAGTC',
       ...
       'GTAACGTCAGACTCGC', 'CAGCCGAAGACGCTTT', 'CGATGTAGTCCGACGT',
       'GTACG

KeyError: 1